# INBOX 
See all incoming messages here

In [ ]:
from s3i import IdentityProvider, Broker, UserMessage, TokenType, Key, Directory
import time 
import base64 
import getpass

In [ ]:
USERNAME = input('[S3I]: Please enter your username:')
PASSWORD = getpass.getpass('[S3I]: Please enter your password:')

In [ ]:
CLIENT = input('[S3I]: Please enter a client id:') 
SECRET = getpass.getpass('[S3I]: Please enter the secret:') 

In [ ]:
IDP = IdentityProvider(grant_type='password', identity_provider_url="https://idp.s3i.vswf.dev/", realm='KWH',
                                             client_id=CLIENT, client_secret=SECRET, username=USERNAME, password=PASSWORD)

In [ ]:
def callback(ch, method, properties, body):
    print("[S3I][" + time.strftime("%Y-%m-%d %H:%M:%S",
                                   time.localtime()) + "]: A new message has been received")
    body_str = body.decode('utf8').replace("'", '"')  # convert bytes to str
    uMsg = UserMessage(msg_blob=body_str)
    
    access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
    dir = Directory(s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)
    """Decrypt the message and verify the signature
    """
    if uMsg.pgpMsg.is_encrypted:
        sec_key = Key(path_demo_dir="key/", filename="workshop_sec.asc")
        #uMsg.decryptAndVerify(sec_key, "", dir)
        uMsg.decrypt(sec_key.key)
        uMsg.convertPgpToMsg()

    print("[S3I]: Subject of the message: " + uMsg.msg["subject"])
    print("[S3I]: Text of the message:  " + uMsg.msg["text"])
    print("[S3I]: Sender of the message:  :", dir.queryThingIDBased(uMsg.msg["sender"]+"/attributes/name"))

    attachments_list = uMsg.msg["attachments"]
    """
    store the attachment file in specified path
    """
    for attachment in attachments_list:
        with open("received_data/"+attachment["filename"], 'wb') as file:
            decode = base64.b64decode(attachment["data"])
            file.write(decode)
            print("[S3I]: Attachment " + attachment["filename"]
               + " of the message is stored in received_data")
        


In [ ]:
access_token = IDP.get_token(TokenType.ACCESS_TOKEN)
broker = Broker(auth_form='Username/Password', username=" ", password=access_token, host="rabbitmq.s3i.vswf.dev")
broker.receive("s3ibs://"+"s3i:bf5a3b54-1ecc-4195-bc55-cb4757b895d0", callback)